In [111]:
import numpy as np
from datetime import datetime, timedelta
import time

In [232]:
# some probabilities should be dynamics, for example:
# buying probability depends on the number of available items
# listing probability increases if user has sold something in the past
# probability of churn increases if user hasn't listed + hasn't bought anything + doesn't have anything in the basket
# instead of using random choise for time, we should use distribution (exponential, binomial, normal etc)
events = {
    'visit': {
        'condition': True,
        'inputs': 'timestamp',
        'time': [0, 20],
        'next_events': ['search', 'list_item', 'do_nothing'],
        'probabilities': [0.6, 0.05, 0.35]
    },
    'create_account': {
        'time': [30, 150],
        'next_events': ['search', 'list_item', 'do_nothing'],
        'probabilities': [0.8, 0.1, 0.1]        
    },
    'list_item': {
        'conditions': ['registered'],
        'time': [90, 300],
        'next_events': ['search', 'list_item', 'do_nothing'],
        'probabilities': [0.1, 0.3, 0.6]        
    },
    'search': {
        'time': [10, 120],
        'next_events': ['search', 'view_item', 'list_item', 'do_nothing'],
        'probabilities': [0.35, 0.5, 0.01, 0.14]       
    },
    'view_item': {
        'time': [10, 30],
        'next_events': ['view_item', 'send_message', 'search', 'add_to_basket', 'list_item', 'do_nothing'],
        'probabilities': [0.4, 0.1, 0.2, 0.1, 0.01, 0.19]          
    },
    'send_message': {
        'conditions': ['registered'],
        'time': [10, 30],
        'next_events': ['view_item', 'search', 'add_to_basket', 'do_nothing'],
        'probabilities': [0.5, 0.25, 0.05, 0.2]          
    },
    'read_message': {
        'conditions': ['n_unread_messages > 0'],
        'time': [1, 10],
        'next_events': ['answer', 'search', 'list_item', 'do_nothing'],
        'probabilities': [0.8, 0.1, 0.01, 0.09]          
    },  
    'answer': {
        'conditions': ['n_read_messages > 0'],
        'time': [5, 120],
        'next_events': ['search', 'list_item', 'do_nothing'],
        'probabilities': [0.3, 0.01, 0.69]          
    }, 
    'add_to_basket': {
        'conditions': ['registered'],
        'time': [5, 120],
        'next_events': ['search', 'view_item', 'open_basket', 'do_nothing'],
        'probabilities': [0.2, 0.2, 0.45, 0.15]        
    },
    'open_basket': {
        'conditions': ['n_items_in_basket > 0'],
        'time': [5, 120],
        'next_events': ['search', 'remove_from_basket', 'pay',  'list_item', 'do_nothing'],
        'probabilities': [0.05, 0.35, 0.45, 0.01, 0.15]        
    },
    'remove_from_basket': {
        'conditions': ['n_items_in_basket > 0'],
        'time': [1, 20],
        'next_events': ['search', 'remove_from_basket', 'pay', 'do_nothing'],
        'probabilities': [0.2, 0.2, 0.2, 0.4]        
    },
    'pay': {
        'conditions': ['registered', 'n_items_in_basket > 0'],
        'time': [180, 1800],
        'next_events': ['search', 'do_nothing'],
        'probabilities': [0.1, 0.9]        
    },
    'do_nothing': {}
}

In [233]:
def create_event_data(event_name, user_id, timestamp, properties=None):
    d = {
        'event_name': event_name,
        'user_id': user_id,
        'timestamp': timestamp
    }
    
    if properties is not None:
        for p in properties.keys():
            d[p] = properties
    
    return d

In [234]:
users = dict()
items = dict()
messages = dict()

In [235]:
class Item:
    def __init__(self, item_id, lister_id, listing_date):
        self.item_id = item_id
        self.lister_id = lister_id
        self.listing_date = listing_date
        self.status = 'active'
        

        

class Message:
    def __init__(self, sender_id, recepient_id, message_id, timestamp):
        self.sender_id = sender_id
        self.recepient_id = recepient_id
        self.message_id = message_id
        self.timestamp = timestamp

In [236]:
current_date = datetime(2021,4,18,23,10,11)

In [237]:
class User:
    def __init__(self, name, user_id):
        self.name = name
        self.user_id = user_id
        self.registered = False

    
    
    satisfaction_impact = {
        'registration': 10,
        'message_sent': 1,
        'message_read': 1,
        'list_item': 10,
        'purchase': 20,
        'sale': 20,
        'delete_item': -20,
        'days_listed': -1,
        'search': -1,
        'item_view': -1
    }
    
    
    
    @property
    def visit_probability(self):
        """Calculate visit_probability as combination of initial probability and satisfaction level and other factor.
        """
        probability_visit_from_satisfaction = 0.01 + self.satisfaction / 1000
        
        if probability_visit_from_satisfaction < 0:
            probability_visit_from_satisfaction = 0
        elif probability_visit_from_satisfaction > 0.05:
            probability_visit_from_satisfaction = 0.05
        
        probability_visit_from_messages = self.n_unread_messages * 0.2
        
        if probability_visit_from_messages > 0.6:
            probability_visit_from_messages = 0.6
        
        probability_visit_total = probability_visit_from_satisfaction + probability_visit_from_messages

        return probability_visit_total
    
    
    
    @property
    def satisfaction(self):
        """Calculate user satisfaction level.
        """
        satisfaction = 0
        
        if self.registered:
            satisfaction += self.satisfaction_impact['registration']
        
        if hasattr(self, 'messages_sent'):
            satisfaction += self.n_messages_sent * self.satisfaction_impact['message_sent']

        if hasattr(self, 'messages_read'):
            satisfaction += self.n_messages_read * self.satisfaction_impact['message_read']

        if hasattr(self, 'n_listed_items'):
            satisfaction += self.n_listed_items * self.satisfaction_impact['list_item']           
        
        if hasattr(self, 'n_purchases'):
            satisfaction += self.n_purchases * self.satisfaction_impact['purchase'] 
        
        if hasattr(self, 'n_sold_items'):
            satisfaction += self.n_sold_items * self.satisfaction_impact['sale'] 

        if hasattr(self, 'item_views'):
            satisfaction += self.item_views * self.satisfaction_impact['item_view'] 
            
        if hasattr(self, 'searches'):
            satisfaction += self.searches * self.satisfaction_impact['search'] 
        
        if hasattr(self, 'n_deleted_items'):
            satisfaction += self.n_deleted_items * self.satisfaction_impact['delete_item'] 
            
        if hasattr(self, 'active_items'):
            for item_id in self.active_items:
                satisfaction += (current_date - items[item_id].listing_date).days * self.satisfaction_impact['days_listed'] 
            
        return satisfaction
    
    

    @property
    def listing_index(self):
        if self.n_sold_items > 0:
            index = self.n_sold_items / self.n_listed_items / 0.5
        elif self.n_listed_items:
            index = 1 - self.n_listed_items * 0.1 if self.n_listed_items <= 10 else 0
        else:
            index = 1
        
        return index

        
    
    @property
    def items_in_basket(self):
        """Calculate items in basket
        """
        return len(self.basket) if hasattr(self, 'basket') else 0
    
    
    
    @property
    def unread_messages(self):
        """Get list of unread messages
        """
        received_messages = self.messages_received if hasattr(self, 'messages_received') else []
        read_messages = self.read_messages if hasattr(self, 'read_messages') else []
        unread_messages = list(set(received_messages) - set(read_messages))
        
        return unread_messages
        
    
    
    @property
    def n_listed_items(self):
        """Calculate number of listed items
        """
        return len(self.listed_items) if hasattr(self, 'listed_items') else 0 
    
    
    
    @property
    def n_active_items(self):
        """Calculate number of active items
        """
        return len(self.active_items) if hasattr(self, 'active_items') else 0 

    
    
    @property
    def n_deleted_items(self):
        """Calculate number of deleted items
        """
        return len(self.deleted_items) if hasattr(self, 'deleted_items') else 0 
    
    
    
    @property
    def n_items_in_basket(self):
        """Calculate number of deleted items
        """
        return len(self.basket) if hasattr(self, 'basket') else 0     
        
 

    @property
    def n_sold_items(self):
        """Calculate number of sold items
        """
        return len(self.sold_items) if hasattr(self, 'sold_items') else 0 
        
 

    @property
    def n_purchases(self):
        """Calculate number of purchased items
        """
        return len(self.purchased_items) if hasattr(self, 'purchased_items') else 0 


    
    @property
    def n_messages_sent(self):
        """Calculate number of sent messages
        """
        return len(self.messages_sent) if hasattr(self, 'messages_sent') else 0     


    
    @property
    def n_messages_received(self):
        """Calculate number of received messages
        """
        return len(self.messages_received) if hasattr(self, 'messages_received') else 0    


    
    @property
    def n_unread_messages(self):
        """Calculate number of received messages
        """
        return len(self.unread_messages) if hasattr(self, 'unread_messages') else 0   

    
    
    @property
    def n_read_messages(self):
        """Calculate number of read messages
        """
        return len(self.messages_read) if hasattr(self, 'messages_read') else 0  
    

    
    def visit(self, platform, country, timestamp):
        """User visit event. 
        It's the first touch with the app within a session.
        Event creates / updates user attributes:
            visits: number of visits.
            last_visit: time of the last visit.
            last_activity: time of the last activity.
            last_properties: properties like platform and country.
        
        Parameters:
            timestamp: time of the event.
            platform: platform of the visit: 'ios', 'android', 'web'.
            country: country code of the visit: 'US', 'DE', 'GB' etc.
        """
        self.active_session = True
        self.last_event = 'visit'
        self.last_activity = timestamp
        self.visits = self.visits + 1 if hasattr(self, 'visits') else 1
        self.last_visit = timestamp
        
        self.last_properties = {
            'platform': platform,
            'country': country
        }
        
        print(self.last_event, timestamp)
    

    
    def create_account(self, timestamp):
        """User creates an account. 
        Parameters:
            timestamp: time of the event.
        """
        self.last_event = 'create_account'
        self.last_activity = timestamp
        self.registered = True
        self.registration_date = timestamp
        
        print(self.last_event, timestamp)
    
  
    
    def send_message(self, timestamp):
        """User sends message to another user. 
        Parameters:
            recepient_id: id of the user who receives the message.
            timestamp: time of the event.
        """
        self.last_event = 'send_message'
        self.last_activity = timestamp
        
        # create message id
        recepient_id = items[self.open_item].lister_id
        message_id = hash(str(self.user_id) + str(recepient_id) + str(timestamp))
        
        # add messages to user attributes
        if hasattr(self, 'messages_sent'):
            self.messages_sent.append(message_id)
        else:
            self.messages_sent = [message_id]
        
        # store data to messages dict
        messages[message_id] = Message(sender_id=self.user_id, 
                                       recepient_id=recepient_id, 
                                       message_id=message_id, 
                                       timestamp=timestamp)      
        
        # update recepient attributes
        if hasattr(users[recepient_id], 'messages_received'):
            users[recepient_id].messages_received.append(message_id)
        else: 
            users[recepient_id].messages_received = [message_id]       
            
        print(self.last_event, timestamp)
        
    
    
    def read_message(self, timestamp):
        """User reads message from another user. 
        Parameters:
            message_id: id of the message.
            timestamp: time of the event.
        """
        self.last_event = 'read_message'
        self.last_activity = timestamp
        
        rand = np.random.default_rng(seed=abs(hash(timestamp)))
        message_id = rand.choice(a=self.unread_messages)
        self.unread_messages.remove(message_id)
        
        # store message to user's read messages
        if hasattr(self, 'read_messages'):
            self.read_messages.append(message_id)
        else:
            self.read_messages = [message_id]
        
        print(self.last_event, timestamp)

        

    def answer(self, timestamp):
        """User reads message from another user. 
        Parameters:
            message_id: id of the message.
            timestamp: time of the event.
        """
        self.last_event = 'answer'
        self.last_activity = timestamp
        
        # get sender_id who will be recepient of the next message
        message_id = self.read_messages[-1]
        recepient_id = messages[message_id].sender_id
        
        # create new message_id
        new_message_id = hash(str(self.user_id) + str(recepient_id) + str(timestamp))

        # add messages to user attributes
        if hasattr(self, 'messages_sent'):
            self.messages_sent.append(new_message_id)
        else:
            self.messages_sent = [new_message_id]
        
        # store data to messages dict
        messages[message_id] = Message(sender_id=self.user_id, 
                                       recepient_id=recepient_id, 
                                       message_id=new_message_id, 
                                       timestamp=timestamp)         
        
        # update recepient attributes
        if hasattr(users[recepient_id], 'messages_received'):
            users[recepient_id].messages_received.append(message_id)
        else: 
            users[recepient_id].messages_received = [message_id]
        
        print(self.last_event, timestamp)
        


    def list_item(self, timestamp):
        """User lists an item. 
        Parameters:
            timestamp: time of the event.
        """
        self.last_event = 'list_item'
        self.last_activity = timestamp
        
        item_id = hash(str(self.user_id) + str(timestamp))
        
        if hasattr(self, 'listed_items'):
            self.listed_items.append(item_id)
        else:
            self.listed_items = [item_id]

        if hasattr(self, 'active_items'):
            self.active_items.append(item_id)
        else:
            self.active_items = [item_id]

        items[item_id] = Item(item_id=item_id, 
                              lister_id=self.user_id, 
                              listing_date=timestamp)
        
        print(self.last_event, timestamp)
        
    
    
    def search(self, timestamp):
        """User performs a search. 
        Parameters:
            timestamp: time of the event.
        """
        self.last_event = 'search'
        self.searches = self.searches + 1 if hasattr(self, 'searches') else 1
        self.last_activity = timestamp
        
        rand = np.random.default_rng(seed=abs(hash(timestamp)))
        self.available_items = rand.choice(a=list(items.keys()), size=20 if len(items.keys())>=20 else len(items.keys()), replace=False)

        print(self.last_event, timestamp)
        
    
        
    def view_item(self, timestamp):
        """User views an item. 
        Parameters:
            timestamp: time of the event.
        """  
        self.last_event = 'view_item'
        self.last_activity = timestamp
        self.item_views = self.item_views + 1 if hasattr(self, 'item_views') else 1
        
        rand = np.random.default_rng(seed=abs(hash(timestamp)))
        item_id = rand.choice(a=self.available_items)
        self.open_item = item_id
        items[item_id].views = items[item_id].views + 1 if hasattr(items[item_id], 'views') else 1
        
        print(self.last_event, timestamp)
    
    
    
    def add_to_basket(self, timestamp):
        """User adds an item to the basket. 
        Parameters:
            timestamp: time of the event.
        """
        self.last_event = 'add_to_basket'
        self.last_activity = timestamp
        
        if hasattr(self, 'basket'):
            self.basket.append(self.open_item)
        else:
            self.basket = [self.open_item]
            
        print(self.last_event, timestamp)

        
        
    def open_basket(self, timestamp):
        """User adds an item to the basket. 
        Parameters:
            timestamp: time of the event.
        """
        self.last_event = 'open_basket'
        self.last_activity = timestamp
            
        print(self.last_event, timestamp)
        
        
    
    def remove_from_basket(self, timestamp):
        """User removes an item to the basket. 
        Parameters:
            timestamp: time of the event.
        """
        self.last_event = 'remove_from_basket'
        self.last_activity = timestamp
        
        rand = np.random.default_rng(seed=abs(hash(timestamp)))
        item_id = rand.choice(a=self.basket)
        self.basket.remove(item_id)
        
        print(self.last_event, timestamp)
        
    
    
    def pay(self, timestamp):
        """User pays for item / set of items. 
        Parameters:
            item_id: id of the item user views.
            timestamp: time of the event.
        """
        self.last_event = 'pay'
        self.last_activity = timestamp
        
        for item_id in self.basket:  
            # updateitems attributes
            items[item_id].status = 'sold'
            items[item_id].buyer = self.user_id
            items[item_id].date_sold = timestamp
        
            # update lister's attributes
            lister_id = items[item_id].lister_id
            users[lister_id].active_items.remove(item_id)
            
            if hasattr(users[lister_id], 'sold_items'):
                users[lister_id].sold_items.append(item_id)
            else:
                users[lister_id].sold_items = [item_id]
        
        # update buyer's attributes
        if hasattr(self, 'purchased_items'):
            self.purchased_items.extend(self.basket)
        else:
            self.purchased_items = self.basket
        
        # empy basket
        self.basket = []
        
        print(self.last_event, timestamp)



    def delete_items(self, item_id, timestamp):
        """User removes an item. 
        Parameters:
            item_id: id of the item user views.
            timestamp: time of the event.
        """
        self.last_event = 'delete_items'
        self.last_activity = timestamp
        self.active_items.remove(item_id)
        items[item_id].status = 'deleted'
        items[item_id].date_deleted = timestamp
        
        if hasattr(self, 'deleted_items'):
            self.deleted_items.append(item_id)
        else:
            self.deleted_items = [item_id]

        
    
    def do_nothing(self, timestamp):
        self.active_session = False

In [238]:
def session(user_id, timestamp):
    if user_id not in users.keys():
        users[user_id] = User(name=str(user_id), user_id='user_id')
    
    users[user_id].visit(timestamp=timestamp, platform='ios', country='DE')
    
    # number of the event
    n = 0
    
    while users[user_id].active_session:
        last_event = users[user_id].last_event
        
        next_events = events[last_event]['next_events'].copy()
        probabilities = events[last_event]['probabilities'].copy()
                        
        
        # adjust registration probability
        if users[user_id].registered == False and 'create_account' not in next_events:
            # add registration as potential event
            next_events.append('create_account')
            probabilities = [prob * 0.8 for prob in probabilities]
            probabilities.append(0.2)

        
        # adjust open basket probability
        if users[user_id].n_items_in_basket > 0 and users[user_id].last_event != 'open_basket' and 'open_basket' not in next_events:
            next_events.append('open_basket')
            probabilities = [prob * 0.8 for prob in probabilities]
            probabilities.append(0.2)
            

        # adjust read_message probability
        if users[user_id].n_unread_messages > 0:
            # add read_message as potential event
            next_events.append('read_message')
            probabilities = [prob * 0.2 for prob in probabilities]
            probabilities.append(0.8)
        
        
        # adjust listing probability
        if 'list_item' in next_events:
            index = next_events.index('list_item')
            probabilities[index] = probabilities[index] * users[user_id].listing_index           
        
        
        # with every event probability of do nothing grows
        if 'do_nothing' in next_events:
            index = next_events.index('do_nothing')
            probabilities[index] = probabilities[index] * (1 + n/100)
            
        
        # check condition for every event
        for event in next_events:
            if 'conditions' in events[event]:
                for condition in events[event]['conditions']:
                    if eval(f'users[user_id].{condition}') == False:
                        index = next_events.index(event)
                        next_events.remove(event)
                        probabilities.pop(index)   
                        break
                        
            
        # normalize probabilities
        total_p = sum(probabilities)
        probabilities = [p/total_p for p in probabilities]
        probabilities[0] = probabilities[0] + 1-sum(probabilities) 
                     
        rand = np.random.default_rng(seed=timestamp.minute*60+timestamp.second+user_id)
        next_event = rand.choice(a=next_events, p=probabilities)
        
        
        time_delta = int(rand.integers(low=events[last_event]['time'][0], high=events[last_event]['time'][1]))
        timestamp = timestamp + timedelta(seconds=time_delta)
        
        eval(f'users[user_id].{next_event}(timestamp=timestamp)')
        n += 1

In [239]:
# create initial set of items
users[1] = User(name='first user', user_id=1)

start_date = datetime.now() - timedelta(days=10)
users[1].create_account(timestamp=start_date)

for i in range(100):
    users[1].list_item(timestamp=start_date + timedelta(seconds=i+1) + timedelta(minutes=i+1))

create_account 2021-04-08 23:11:37.686658
list_item 2021-04-08 23:12:38.686658
list_item 2021-04-08 23:13:39.686658
list_item 2021-04-08 23:14:40.686658
list_item 2021-04-08 23:15:41.686658
list_item 2021-04-08 23:16:42.686658
list_item 2021-04-08 23:17:43.686658
list_item 2021-04-08 23:18:44.686658
list_item 2021-04-08 23:19:45.686658
list_item 2021-04-08 23:20:46.686658
list_item 2021-04-08 23:21:47.686658
list_item 2021-04-08 23:22:48.686658
list_item 2021-04-08 23:23:49.686658
list_item 2021-04-08 23:24:50.686658
list_item 2021-04-08 23:25:51.686658
list_item 2021-04-08 23:26:52.686658
list_item 2021-04-08 23:27:53.686658
list_item 2021-04-08 23:28:54.686658
list_item 2021-04-08 23:29:55.686658
list_item 2021-04-08 23:30:56.686658
list_item 2021-04-08 23:31:57.686658
list_item 2021-04-08 23:32:58.686658
list_item 2021-04-08 23:33:59.686658
list_item 2021-04-08 23:35:00.686658
list_item 2021-04-08 23:36:01.686658
list_item 2021-04-08 23:37:02.686658
list_item 2021-04-08 23:38:03.686

In [240]:
# create events for the first users
for i in range(2,101):
    print('\nUSER: {}'.format(i))
    users[i] = User(name='{} user'.format(i), user_id=i)
    session(user_id=i, timestamp=start_date + timedelta(minutes=300+i) + timedelta(seconds=i))


USER: 2
visit 2021-04-09 04:13:39.686658
create_account 2021-04-09 04:13:42.686658
search 2021-04-09 04:15:55.686658
view_item 2021-04-09 04:17:19.686658

USER: 3
visit 2021-04-09 04:14:40.686658

USER: 4
visit 2021-04-09 04:15:41.686658

USER: 5
visit 2021-04-09 04:16:42.686658
search 2021-04-09 04:16:56.686658
view_item 2021-04-09 04:17:58.686658
search 2021-04-09 04:18:12.686658
view_item 2021-04-09 04:19:33.686658
view_item 2021-04-09 04:19:52.686658

USER: 6
visit 2021-04-09 04:17:43.686658
create_account 2021-04-09 04:17:51.686658

USER: 7
visit 2021-04-09 04:18:44.686658

USER: 8
visit 2021-04-09 04:19:45.686658
search 2021-04-09 04:19:50.686658
view_item 2021-04-09 04:20:07.686658
view_item 2021-04-09 04:20:19.686658
search 2021-04-09 04:20:42.686658
view_item 2021-04-09 04:22:38.686658
create_account 2021-04-09 04:23:04.686658
search 2021-04-09 04:23:49.686658
search 2021-04-09 04:25:38.686658
search 2021-04-09 04:26:51.686658

USER: 9
visit 2021-04-09 04:20:46.686658
search 

In [243]:
for user_id in users.keys():
    if user_id != 1:
        print(f'user_id: {user_id}, satisfaction: {users[user_id].satisfaction}, visit probability: {users[user_id].visit_probability}')

user_id: 2, satisfaction: 8, visit probability: 0.018000000000000002
user_id: 3, satisfaction: 0, visit probability: 0.01
user_id: 4, satisfaction: 0, visit probability: 0.01
user_id: 5, satisfaction: -5, visit probability: 0.005
user_id: 6, satisfaction: 10, visit probability: 0.02
user_id: 7, satisfaction: 0, visit probability: 0.01
user_id: 8, satisfaction: 2, visit probability: 0.012
user_id: 9, satisfaction: -3, visit probability: 0.007
user_id: 10, satisfaction: 0, visit probability: 0.01
user_id: 11, satisfaction: 0, visit probability: 0.01
user_id: 12, satisfaction: 0, visit probability: 0.01
user_id: 13, satisfaction: 11, visit probability: 0.020999999999999998
user_id: 14, satisfaction: -2, visit probability: 0.008
user_id: 15, satisfaction: 0, visit probability: 0.01
user_id: 16, satisfaction: 1, visit probability: 0.011
user_id: 17, satisfaction: 0, visit probability: 0.01
user_id: 18, satisfaction: -4, visit probability: 0.006
user_id: 19, satisfaction: 6, visit probabilit

In [219]:
users[31].registered

False

## Writing data to bigquery

In [27]:
from google.cloud import storage
from google.cloud import bigquery

import sys
import os

In [44]:
bigquery_client = bigquery.Client.from_service_account_json('../../credentials/data-analysis-sql-309220-6ce084250abd.json')

In [41]:
countries = ['UK', 'DE', 'AT']
countries_probs = [0.5, 0.4, 0.1]

agents = ['android', 'ios', 'web']
agents_probs = [0.4, 0.3, 0.3]

rand = np.random.default_rng(seed=1)

objects = []
for i in range(1000):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    object = {
        'timestamp': timestamp,
        'id': str(hash(timestamp)),
        'nested': {
            'os': rand.choice(a=agents, p=agents_probs),
            'country': rand.choice(a=countries, p=countries_probs)
        }
    }
    
    objects.append(object)
    
    time.sleep(0.01)

In [45]:
bq_error = bigquery_client.insert_rows_json('data-analysis-sql-309220.synthetic.nested_test', objects)
if bq_error != []:
    print(bq_error) 